In [ ]:
import json
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import numpy as np

In [ ]:
def save(model, filename: str, model_name: str):
    """Saves the model to an .h5 file and the model name to a .json file.

    Args:
        model: Model to be saved
        filename: Relative path to the file without the extension.
        
    """
    # Save Keras model
    model.save(filename + '.h5')

    # Save base model information
    with open(filename + '.json', 'w', encoding='utf-8') as f:
        json.dump(model_name, f, ensure_ascii=False, indent=4, sort_keys=True)

In [ ]:
train_datagen = ImageDataGenerator(
        # reducing/normalizing the pixels
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2) 
#We dont want the dataset to be flipped as orientation is important, so we delete the horizontal_flip parameter
#connecting the image augmentation tool to our dataset

targetSizeWidth = 112
targetSizeHeight = 112
targetSize = (targetSizeWidth, targetSizeHeight)

train_set = train_datagen.flow_from_directory(
        './training',
        target_size = targetSize,
        batch_size=32,
        class_mode='categorical')


#only rescaling but no transformations
test_datagen = ImageDataGenerator(rescale=1./255)
#connecting to the test data
test_set = test_datagen.flow_from_directory(
        './test',
        target_size = targetSize,
        batch_size = 32,
        class_mode = 'categorical')

print(test_set)

In [ ]:
#--------------------- Building CNN --------------------#
# initializing CNN as sequential layers
from tf.keras.callbacks import EarlyStopping
def create_model():
        
    cnn = tf.keras.models.Sequential()

    # Step 1: Convolution to get the Feature Map
    cnn.add(tf.keras.layers.Conv2D(filters= 210 , kernel_size = 2, activation = 'relu', input_shape=[640,480,3]))
    # filters: output feature map
    # kernel_size: size of the feature detector
    # strides: step size from one filter to the next default is 1
    # Step 2: Max Pooling
    cnn.add(tf.keras.layers.MaxPool2D(pool_size=2 ,strides=2))
    #adding a second convolutional layer
    cnn.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, activation = 'relu'))
    cnn.add(tf.keras.layers.MaxPool2D(pool_size=2 ,strides=2))
    #adding a second convolutional layer
    cnn.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, activation = 'relu'))
    cnn.add(tf.keras.layers.MaxPool2D(pool_size=2 ,strides=2))
    #adding a second convolutional layer
    cnn.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, activation = 'relu'))
    cnn.add(tf.keras.layers.MaxPool2D(pool_size=2 ,strides=2))
    # Step 3: Flattening
    cnn.add(tf.keras.layers.Flatten())

    # Step 4: Full Connection
    cnn.add(tf.keras.layers.Dense(units = 256, activation = 'relu'))

    # Step 5: Output Layer
    cnn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))
    return cnn

#--------------------- Training the CNN --------------------#
#compiling the CNN
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
#training the CNN on the training set and evaluating it on the test set
cnn.fit(x = train_set, validation_data = test_set, epochs = 25)

early_stopping_monitor = EarlyStopping(patience=10)
history = model.fit(X, y, validation_split=0.33, epochs=200, batch_size=15, verbose=0, callbacks=[early_stopping_monitor])
